In [995]:
from operator import itemgetter
import pandas as pd
import fitz
import json
from bs4 import BeautifulSoup
import itertools
def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles


In [1287]:
def unlevel(obj):
    while isinstance(obj, list) and len(obj) == 1:
        obj = obj[0]
    if isinstance(obj, list):
        return [unlevel(item) for item in obj]
    else:
        return obj

In [996]:
import camelot

In [997]:
doc = fitz.open('/home/jupyter/fpr_pdf3.pdf')

In [998]:
import fitz  # this is pymupdf

with fitz.open('/home/jupyter/fpr_pdf3.pdf') as doc:
    text = ""
    for page in doc:
        text += page.getText()
        #print(text)

#print(text)

In [999]:
doc = fitz.open('/home/jupyter/fpr_pdf3.pdf')

In [1000]:
font_counts, styles = fonts(doc, granularity=False)

In [1001]:
font_counts, styles

([('9.960000038146973', 724),
  ('9.0', 18),
  ('14.039999961853027', 6),
  ('12.960000038146973', 4),
  ('11.039999961853027', 3),
  ('6.480000019073486', 2),
  ('21.959999084472656', 1)],
 {'21.959999084472656': {'size': 21.959999084472656, 'font': 'ArialMT'},
  '9.960000038146973': {'size': 9.960000038146973, 'font': 'ArialMT'},
  '12.960000038146973': {'size': 12.960000038146973, 'font': 'ArialMT'},
  '14.039999961853027': {'size': 14.039999961853027, 'font': 'ArialMT'},
  '9.0': {'size': 9.0, 'font': 'ArialMT'},
  '6.480000019073486': {'size': 6.480000019073486, 'font': 'ArialMT'},
  '11.039999961853027': {'size': 11.039999961853027,
   'font': 'BCDFEE+Calibri'}})

In [1002]:
def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag 
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

In [1003]:
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text
                # REMEMBER: multiple fonts and sizes are possible IN one block
                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)
    return header_para


In [1004]:
#doc = fitz.open('/home/jupyter/FPR_281020_3.pdf')
document = '/home/jupyter/fpr_pdf3.pdf'
doc = fitz.open(document)
toc = doc.getToC()

font_counts, styles = fonts(doc, granularity=False)

size_tag = font_tags(font_counts, styles)

elements = headers_para(doc, size_tag)

with open("doc2.json", 'w') as json_out:
    json.dump(elements, json_out)

In [1005]:
tables = camelot.read_pdf(document , flavor='stream')

In [1006]:
noway = tables[2].df

#df.to_csv()

In [1007]:
noway

,0,1
0,xisting policy,
1,This will Field_15 the following Field_16:,
2,R\nepeat for multiple policies,
3,Name,Elnaz Nobari
4,Product,potatoe Level Life Cover tomatoe
5,Sum assured,£13
6,Term,200 years 10 millenium
7,Premium,£400 a month
8,In trust,Yes
9,Guaranteed insurability,No


In [1008]:
way = noway[noway[1] != '']
way.columns = ["Field","Value"]
print(way)

                     Field                             Value
3                     Name                      Elnaz Nobari
4                  Product  potatoe Level Life Cover tomatoe
5              Sum assured                               £13
6                     Term            200 years 10 millenium
7                  Premium                      £400 a month
8                 In trust                               Yes
9  Guaranteed insurability                                No


### fitz.Document.getXmlMetadata('/home/jupyter/FPR_281020_3.pdf')

In [1009]:
#!/usr/bin/python

#from bs4 import BeautifulSoup

with open('doc2.json', 'r') as f:

    contents = f.read()

    soup = BeautifulSoup(contents, 'lxml')

    #print(soup.find('ul', attrs={ 'id' : 'mylist'}))
    #print(soup.find('ul', id='h1'))
    root = soup.body
        #root = soup.body
    for tag in soup.find_all('h1'):
        out_name = f'{tag.name}: {tag.text}'
    #root_childs = [e.name for e in root.descendants if e.name is not None]
    #print(root_childs)
out_name

'h1: My protection recommendations |", "", "'

In [1010]:
#contents

In [1011]:
page = doc[3]

In [1012]:
#g = page.getTextPage()

In [1013]:
#g2 = g.extractBLOCKS(3)

In [1014]:
k2 = page.getText("dict")
#doc.TextPage.extractText()

In [1015]:
#k2

In [1016]:
for p in k2['blocks']:
    #print( p['bbox'])
    if p['type'] == 0:
        for pp in p['lines']:
            for ppp in pp['spans']:
                print(ppp['text'])
                print(ppp['bbox'])
        #print(pp['spans']['bbox'])
    #print(p)
    #print(p['label'])
    #print(p['words'])
    #print(p['linking'])
    #print(p['id'])

    #print('')

Lifestyle - critical illness 
(72.02400207519531, 70.32801055908203, 211.4928741455078, 88.13504791259766)
Lifestyle protection for you 
(72.02400207519531, 86.08699035644531, 239.39312744140625, 105.3779525756836)
Summarise the customer’s needs and personal reasons for reviewing this area. Keep concise and 
(72.02400207519531, 112.68348693847656, 507.31427001953125, 123.81067657470703)
take care not to repeat information from elsewhere in the report.  
(72.02400207519531, 123.35299682617188, 359.9588928222656, 137.0380401611328)
INSERT_39 
(72.02400207519531, 135.71298217773438, 128.18887329101562, 149.3980255126953)
Repeat the following product details for each recommended policy. Ensure all details match the 
(72.02400207519531, 148.19296264648438, 495.307861328125, 161.8780059814453)
illustration(s). For joint cases, add customer names if applicable. 
(72.02400207519531, 160.55300903320312, 359.4788818359375, 174.23805236816406)
Name 
(77.42400360107422, 173.08279418945312, 107.428

In [1017]:
import zipfile

In [1018]:
#soup = BeautifulSoup(xml_data, 'xml')
docx2 = 'Protection recommendations_1.docx'
document2 = zipfile.ZipFile(docx2)
xml_data = document2.read('word/document.xml')
soup = BeautifulSoup(xml_data, 'xml')

In [1019]:
#soup

In [1020]:
#with open("docsoup.json", 'w') as json_out:
    #json.dump(soup, json_out)

In [1021]:
#xml_input = open(k2,"r",encoding="utf-8")

ocr_lines = soup.findAll("span", {"class": "ocr_line"})
#We will save coordinates of line and the text contained in the line in lines_structure list
lines_structure = []
for line in ocr_lines:
    line_text = line.text.replace("\n"," ").strip()
    title = line['title']
    #The coordinates of the bounding box
    x1,y1,x2,y2 = map(int, title[5:title.find(";")].split())
    lines_structure.append({"x1":x1,"y1":y1,"x2":x2,"y2":y2,"text": line_text})

In [1022]:
lines_structure

[]

In [1023]:
import bs4
#xml_input = open("output_hocr.hocr","r",encoding="utf-8")
soup = bs4.BeautifulSoup(kxml,'lxml')
ocr_lines = soup.findAll("span", {"class": "ocr_line"})
#We will save coordinates of line and the text contained in the line in lines_structure list
lines_structure = []
for line in ocr_lines:
    line_text = line.text.replace("\n"," ").strip()
    title = line['title']
    #The coordinates of the bounding box
    x1,y1,x2,y2 = map(int, title[5:title.find(";")].split())
    lines_structure.append({"x1":x1,"y1":y1,"x2":x2,"y2":y2,"text": line_text})

NameError: name 'kxml' is not defined

In [1024]:
lines_structure

[]

In [1025]:
with open('/home/jupyter/doc.json') as json_file:
    data = json.load(json_file)

In [1026]:
datpdf = pd.DataFrame.from_dict(data)

In [1027]:
datpdf.head(50)

,0
0,||
1,|
2,<s2>HSBC UK Bank plc. Registered in England & ...
3,<s3>Authorised by the Prudential Regulation Au...
4,
5,<h1>Financial Planning Report |
6,
7,<h5>Elnaz Nobari ||||||||||||||||||
8,<p>Prepared by || INSERT adviser name and post...
9,|


In [1028]:
p = ('/home/jupyter/FPR_01112020_all.xlsm')
df = pd.read_excel(
    p ,header = None
)

In [1029]:
len(df.columns)

12

In [1030]:
for i in range(0,len(df.columns)-1):
    df[i] = df[i].str.replace("\r", "")
#df.applymap(lambda x: x.replace("\r", ""))

In [1031]:
UniqueNames = df[2].unique()

In [1032]:
def unique_non_null(s):
    return s.dropna().unique()

In [1033]:
uns = unique_non_null(df[2])

In [1034]:
uns

array(['Introduction', 'My protection recommendations',
       'Money for investment', 'My SELECT recommendations',
       'My estate planning recommendations', 'Your agreed risk level',
       'Reason for your agreed risk level', 'SELECT information',
       'Other financial planning areas', 'My other recommendations ',
       'What this will cost you', 'Risks and implications ',
       'What else you need to know', 'Documents you should read',
       'Next steps'], dtype=object)

In [1035]:
import docx
from bs4 import BeautifulSoup
import zipfile

In [1265]:
docx2 = 'Protection recommendations_1.docx'

In [1266]:
bestand = docx.Document(docx2)
tabellen = bestand.tables

In [1267]:
alletabellen = []     
for i, tabel in enumerate(tabellen):
    for row in tabellen[i].rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                alletabellen.append(paragraph.text)

#get data from all the dropdown lists
import lxml.etree
docx2 = 'Protection recommendations_1.docx'
document2 = zipfile.ZipFile(docx2)
xml_data = document2.read('word/document.xml')

tree = lxml.etree.fromstring(xml_data)
document2.close()

In [1268]:
from docx.api import Document
document = Document('Protection recommendations_1.docx')
table = document.tables

In [1269]:
import docx
 
# open connection to Word Document
doc = docx.Document('Protection recommendations_1.docx')
 
# read in each paragraph in file
result = [[p.text] for p in doc.paragraphs]

In [1270]:

#result

In [1271]:
from docx2python import docx2python
 
# extract docx content
doc_result = docx2python('Protection recommendations_1.docx')
#doc_result.body
#doc_result.tables

In [1272]:
pd.DataFrame(doc_result.body[6][1:])

""


In [1273]:
pd.DataFrame(doc_result.body[1][1:]).\
                            applymap(lambda val: val[0].strip("\t"))

,0,1
0,Product,S_Field_1_0 Field_1 policy type
1,Field_2,Field_3
2,Term,Field_4
3,Premium,£INSERT_4 a month
4,Payable on,Field_5
5,Trust recommended,Field_6


In [1274]:
import pandas as pd
pd.DataFrame(doc_result.body[1][1:])

,0,1
0,[Product],[S_Field_1_0 Field_1 policy type]
1,[Field_2],[Field_3]
2,[Term],[Field_4]
3,[Premium],[£INSERT_4 a month]
4,[Payable on],[Field_5]
5,[Trust recommended],[Field_6]


In [1275]:
from bs4 import BeautifulSoup
from html import escape
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.__t = 0
        self.lines = []
        self.__current_line = ''
        self.__current_tag = ''

    @staticmethod
    def __attr_str(attrs):
        return ' '.join('{}="{}"'.format(name, escape(value)) for (name, value) in attrs)

    def handle_starttag(self, tag, attrs):
        if tag != self.__current_tag:
            self.lines += [self.__current_line]

        self.__current_line = '\t' * self.__t + '<{}>'.format(tag + (' ' + self.__attr_str(attrs) if attrs else ''))
        self.__current_tag = tag
        self.__t += 1

    def handle_endtag(self, tag):
        self.__t -= 1
        if tag != self.__current_tag:
            self.lines += [self.__current_line]
            self.lines += ['\t' * self.__t + '</{}>'.format(tag)]
        else:
            self.lines += [self.__current_line + '</{}>'.format(tag)]

        self.__current_line = ''

    def handle_data(self, data):
        self.__current_line += data

    def get_parsed_string(self):
        return '\n'.join(l for l in self.lines if l)


In [1276]:
parser = MyHTMLParser()
#print('custom html parser:')
#print('*' * 80)
parser.feed(str(soup))
#print(parser.get_parsed_string())

In [1277]:
soup = BeautifulSoup(xml_data, 'xml')
gegevens = soup.findAll('listItem')     #search dropdownlists (n = 12)

dropdownlist = []
dropdownlistdata = []

#for i in gegevens:
    #print(i)#dropdownlist.append(i.find('value'))

In [1278]:
cbox = []
pbox = []
divs = soup.find_all('sdt')
for div in divs:
    kaleh = div('sdtPr')
    pacheh = div('sdtContent')
    if (len(kaleh) == len(pacheh)):
        for i in range(0,len(kaleh)):
            k1 = kaleh[i]('comboBox')
            p1 = pacheh[i]('r')
            if k1!=[] and  p1!=[] :
                for kav in k1:
                    k2 =  [num["w:value"] for num in kav]
                    cbox.append(k2)
###here p1[0] was chosen as manual inspection 
                for kp in p1:
                    for div in kp('t'):
                        pres = (div.text)
                        pbox.append(pres)


In [1279]:
dropdown_result = [list(x) for x in zip(cbox, pbox)]

In [1280]:
for res22 in dropdown_result:
    res22[0].pop(0)

In [1281]:
dropdown_result[33]

[['Yes', 'No'], 'Field_18']

In [1282]:
from fuzzywuzzy import fuzz

In [1283]:
divs = soup.find_all('tbl')

In [1284]:
#divs

In [1417]:
k_tables = []
k_all_tbl = []
divs = soup.find_all('tbl')
#print(divs)
for i in range(0,len(divs)):
    k_tables = []
    tbl = divs[i]
    #print(tbl, "____")
    rows = divs[i]('tr')
    #print(rows)
    for ii in range(0, len(divs[i]('tr'))):
        #row in rows:
        columns = divs[i]('tr')[ii]('tc')
        #print(columns)
        k2 =  [[num2.contents for num2 in num("t") ] for num in columns]
        k_tables.append(k2)
    #print(k_tables,"___________________________")
    k_all_tbl.append(k_tables)
#k_all_tbl

In [1418]:
#divs[1]('tr')[1]('tc')[1]('t')[4].contents
k_a2 = unlevel(k_all_tbl)

In [1419]:
k_a2[4]

[['Name', 'INSERT_67'],
 ['Product', ['S_Field_50_49', ' Income Protection Policy']],
 ['Benefit amount',
  ['£',
   'INSERT_68',
   ' a month payable after ',
   'INSERT_69',
   ' ',
   'Field_42',
   'Multiple',
   ' £',
   'INSERT_70',
   ' a month payable after ',
   'INSERT_71',
   ' ',
   'Field_43']],
 ['Term', 'Field_44'],
 ['Premium', ['£', 'INSERT_72', ' a month']]]

In [1420]:
k_all_tbl[-1][0]
k_all_tbl[-2]

[[[['Name']], [['INSERT_113']]],
 [[['Product']],
  [['INSERT_114'], [' '], ['S_Field_87_86'], [' '], ['INSERT_115']]],
 [[['Sum assured']], [['£'], ['INSERT_116']]],
 [[['Term']], [['INSERT_117'], [' years '], ['INSERT_118']]],
 [[['Premium']], [['£'], ['INSERT_119'], [' a month']]],
 [[['In trust']], [['Field_73']]]]

In [1421]:
#k_all_tbl = unlevel(k_all_tbl)
if len(k_all_tbl[-1]) ==1 :
    k_all_tbl[-2].append(k_all_tbl[-1][0])
#k_all_tbl.remove(k_all_tbl[-1])
k_all_tbl.remove(k_all_tbl[-1])
k_all_tbl
print("CHECK WHY THERE is row 1 size table")

CHECK WHY THERE is row 1 size table


In [1422]:
k_all_tbl

[[[[['Name']], [['INSERT_3']]],
  [[['Product']], [['S_Field_1_0'], [' '], ['Field_1 policy type']]],
  [[['Field_2']], [['Field_3']]],
  [[['Term']], [['Field_4']]],
  [[['Premium']], [['£'], ['INSERT_4'], [' a month']]],
  [[['Payable on']], [['Field_5']]],
  [[['Trust recommended']], [['Field_6']]]],
 [[[['Name']], [['INSERT_9']]],
  [[['Product']],
   [['INSERT_10'], [' '], ['S_Field_19_18'], [' '], ['INSERT_11']]],
  [[['Sum assured']], [['£'], ['INSERT_12']]],
  [[['Term']], [['INSERT_13'], [' years '], ['INSERT_14']]],
  [[['Premium']], [['£'], ['INSERT_15'], [' a month']]],
  [[['In trust']], [['Field_17']]],
  [[['Guaranteed insurability option available']], [['Field_18']]]],
 [[[['Name']], [['INSERT_40']]],
  [[['Product']], [['S_Field_34_33'], [' Level Critical Illness Cover']]],
  [[['Sum assured']], [['£'], ['INSERT_41']]],
  [[['Term']], [['Field_30']]],
  [[['Premium']], [['£'], ['INSERT_42'], [' a month']]]],
 [[[['Name']], [['INSERT_46']]],
  [[['Product']], [['INSERT_

In [1423]:
#def unlistmyhart(l):
    #if len(l) ==1:
    #while isinstance(l, list) == True:
       # if len(l) ==1:
           # l = l[0]
            #continue
        #if len(l)>1:
            #for ik in range(0,len(l)):
               # if isinstance(l[ik], list) == True :
                        #l[ik] = unlistmyhart(l[ik])
                        #print(l)
                        #continue
               
        #else:
                    #break
                        #or len(l)>1:
                    
            #break
    #return(l)

In [1424]:
def product(llist):
    result = [[]]

    for lst in llist:
        result = [x + [y] for x in result for y in lst]

    return result

def product2(llist):
    result = [[]]
    for lst in llist:
        new_result = [] # let's manually build out new result
        for existing in result:            
            for element in lst:
                new_result.append(existing + [element])
        result = new_result # update result for next lst
    return result

In [1425]:
k_all_tbl[0][1][1][0]

['S_Field_1_0']

In [1428]:
for ik in range(0,len(k_all_tbl)):
    #print(1)
    #loop through lists inside nested list -if single element list is nested, remove list before comparison, if more than 1 element keep list, and iterativley compare
    for ikk in range(0,len(k_all_tbl[ik])):
        for ikkk in range(0,(len (k_all_tbl[ik][ikk]))):
                for res1 in dropdown_result:
                    k_all_tbl[ik][ikk][ikkk] = unlevel(k_all_tbl[ik][ikk][ikkk])
                    if k_all_tbl[ik][ikk][ikkk] == res1[1]:
                        k_all_tbl[ik][ikk][ikkk] = res1[0]
                        
                            
                                    
                                
                        #print((k_all_tbl[ik][ikk][ikkk][ikkkk]))
                    if len(k_all_tbl[ik][ikk][ikkk]) >1:
                        for ikkkk in range(0,(len (k_all_tbl[ik][ikk][ikkk]))):
                                    if k_all_tbl[ik][ikk][ikkk][ikkkk] == res1[1]:
                                        k_all_tbl[ik][ikk][ikkk][ikkkk] = res1 [0]
                                        print(product(k_all_tbl[ik][ikk][ikkk]))
                                        
                                        #new_result=[]
                                        #for jj in range(0,len(k_all_tbl[ik][ikk][ikkk][ikkkk])):
                                            #new_result=[]
                                            #print(k_all_tbl[ik][ikk][ikkk][ikkkk][jj])
                                            #new_result = k_all_tbl[ik][ikk][ikkk][ikkkk]
                                            #new_result = k_all_tbl[ik][ikk][ikkk][ikkkk][jj]
                                            #print(new_result)
                                            #k_all_tbl[ik][ikk][ikkk].append(new_result)


In [1429]:
k_all_tbl[0][1][1]

[['Zurich', 'Aviva', 'HSBC', 'Zurich'],
 ' ',
 ['Level Life Cover', 'Family Income Benefit', 'Family Income Benefit'],
 'Zurich',
 'Aviva',
 'HSBC',
 'Zurich',
 'Level Life Cover',
 'Family Income Benefit',
 'Family Income Benefit']

In [1336]:
#(k_all_tbl[0][1][1][0]) == dropdown_result[7][1]

In [1337]:
#dropdown_result[7][1]

In [1338]:
#dropdown_result

In [1339]:
#k_all_tbl.replace({'[':''}, regex=True)

In [1341]:
way

,Field,Value
3,Name,Elnaz Nobari
4,Product,potatoe Level Life Cover tomatoe
5,Sum assured,£13
6,Term,200 years 10 millenium
7,Premium,£400 a month
8,In trust,Yes
9,Guaranteed insurability,No


In [1353]:
df = pd.DataFrame()
dataframe = []
#dataframe.append(None)
for ilk in range(0,len(k_all_tbl)):
    
    df = pd.DataFrame(k_all_tbl[ilk])
    df.columns = ['Field','Value']
    for cols in df:
        for irow in range(0,len(df[cols])):
            #while isinstance(df[cols][irow], list) == True and len(df[cols][irow])==1:
                df[cols][irow] = unlevel(df[cols][irow])
                   # continue
                    #if isinstance(df[cols][irow], list) == False or len(df[cols][irow])>1 :
                        #break
    dataframe.append(df)

In [1343]:
dataframe[2]

,Field,Value
0,Name,INSERT_40
1,Product,"[[Aegon, Aviva Core, Aviva Upgraded+, HSBC], ..."
2,Sum assured,"[£, INSERT_41]"
3,Term,"[ii_Field_362_1 years, to age ii_Field_363_2]"
4,Premium,"[£, INSERT_42, a month]"


In [1344]:
import itertools
import collections
from collections.abc import Iterable

In [1345]:
list_df = dataframe[2].values.tolist()
flatten = lambda *n: (e for a in n
        for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

#l1 = (list(flatten(list_df)))
list_df
    

[['Name', 'INSERT_40'],
 ['Product',
  [['Aegon', 'Aviva Core', 'Aviva Upgraded+', 'HSBC'],
   ' Level Critical Illness Cover']],
 ['Sum assured', ['£', 'INSERT_41']],
 ['Term', ['ii_Field_362_1 years', 'to age ii_Field_363_2']],
 ['Premium', ['£', 'INSERT_42', ' a month']]]

In [1329]:
target = way.values.tolist()
for i in range(0,len(dataframe)):
    list_df=[]
    
    list_df = dataframe[i].values.tolist()
    flatten = lambda *n: (e for a in n
        for e in (flatten(*a) if isinstance(a, (tuple, list)) else (a,)))

    l1 = (list(flatten(list_df)))
    l2 = list(flatten(target))
    var1 = set(l1) & set(l2)
    print(var1)

{'Yes', 'Name', 'Sum assured', 'Premium', 'No', 'Product', 'Term'}
{'Yes', 'Name', 'Sum assured', 'Premium', 'In trust', 'No', 'Product', 'Term'}
{'Sum assured', 'Name', 'Premium', 'Term', 'Product'}
{'Yes', 'Name', 'Sum assured', 'Premium', 'No', 'Product', 'Term'}
{'Term', 'Name', 'Product', 'Premium'}
{'Yes', 'Name', 'Premium', 'No', 'Product', 'Term'}
{'Yes', 'Name', 'Sum assured', 'Premium', 'No', 'Product', 'Term'}
{'Yes', 'Name', 'Sum assured', 'Premium', 'In trust', 'No', 'Product', 'Term'}


In [1354]:
dataframe[1].Value[1]

['INSERT_10',
 ' ',
 ['Level Life Cover', 'Family Income Benefit', 'Whole of Life Cover'],
 ' ',
 'INSERT_11']

In [1355]:
for elem in range(0,len(dataframe[1].Value[1])):
    if isinstance(dataframe[1].Value[1][elem], list) == False :
        dataframe[1].Value[1][elem] = list.append(dataframe[1].Value[1][elem])
dataframe[1].Value[1]

TypeError: descriptor 'append' requires a 'list' object but received a 'NavigableString'

In [1298]:
product(dataframe[1].Value[1])

[['I', ' ', 'Level Life Cover', ' ', 'I'],
 ['I', ' ', 'Level Life Cover', ' ', 'N'],
 ['I', ' ', 'Level Life Cover', ' ', 'S'],
 ['I', ' ', 'Level Life Cover', ' ', 'E'],
 ['I', ' ', 'Level Life Cover', ' ', 'R'],
 ['I', ' ', 'Level Life Cover', ' ', 'T'],
 ['I', ' ', 'Level Life Cover', ' ', '_'],
 ['I', ' ', 'Level Life Cover', ' ', '1'],
 ['I', ' ', 'Level Life Cover', ' ', '1'],
 ['I', ' ', 'Family Income Benefit', ' ', 'I'],
 ['I', ' ', 'Family Income Benefit', ' ', 'N'],
 ['I', ' ', 'Family Income Benefit', ' ', 'S'],
 ['I', ' ', 'Family Income Benefit', ' ', 'E'],
 ['I', ' ', 'Family Income Benefit', ' ', 'R'],
 ['I', ' ', 'Family Income Benefit', ' ', 'T'],
 ['I', ' ', 'Family Income Benefit', ' ', '_'],
 ['I', ' ', 'Family Income Benefit', ' ', '1'],
 ['I', ' ', 'Family Income Benefit', ' ', '1'],
 ['I', ' ', 'Whole of Life Cover', ' ', 'I'],
 ['I', ' ', 'Whole of Life Cover', ' ', 'N'],
 ['I', ' ', 'Whole of Life Cover', ' ', 'S'],
 ['I', ' ', 'Whole of Life Cover', ' ', 'E'

In [1218]:
dataframe[1].Value[2]

['£', 'INSERT_12']

In [1219]:
dataframe[1].Value[3]

['INSERT_13', ' years ', 'INSERT_14']

In [1220]:
dataframe[1].Value[4]

['£', 'INSERT_15', ' a month']

In [1221]:
dataframe[1].Value[5]

['Yes', 'No']

In [1223]:
dataframe[0].Field

0                             Name
1                          Product
2    [Sum assured, Benefit amount]
3                             Term
4                          Premium
5                       Payable on
6                Trust recommended
Name: Field, dtype: object

In [1224]:
dataframe[0].Value

0                                             INSERT_3
1    [[Aegon, Aviva, HSBC, Zurich],  , [Level Life ...
2             [£ii_Field_52_1, £ii_Field_53_2 a month]
3    [ii_Field_62_1 years, to age ii_Field_63_2, Wh...
4                              [£, INSERT_4,  a month]
5                                 [Death, First death]
6                                            [Yes, No]
Name: Value, dtype: object

In [1201]:
template_df =[]
myset1 = set(way.Field)
#print(myset1)
pdf_list = way.Field #list(myset1)
inp_tbl = way.Field #pdf_list
for i in range(0, len(dataframe)):

    tmpl_list=[]
    tmpl_list = list(dataframe[i].Field)

    #for gg in range(0,len(tmpl_list)):
        #if isinstance(tmpl_list[gg], list) == False:
            #tmpl_list[gg]= list(tmpl_list[gg])



    #lst = [['a'], ['b', 'c'], ['d', ['e', 'f']]]
    outp = list(itertools.product(tmpl_list))
    #print(outp)
    out = []
    for i in outp:
        temp = []
        for j in i:
            #print(j)
            if isinstance(j, list):
                for k in j:
                    temp.append(k)
            else:
                temp.append(j)
        out.append(temp)
    #print(out)

    tmpl_list2  = out

    outp = list(itertools.product(*tmpl_list2))
    #print(outp)
    out = []
    for i in outp:
        temp = []
        for j in i:
            if isinstance(j, list):
                for k in j:
                    temp.append(k)
            else:
                temp.append(j)
        out.append(temp)
    template_df.append(out) 
    #return template_df

In [1213]:
template_df[0]

[['Name',
  'Product',
  'Sum assured',
  'Term',
  'Premium',
  'Payable on',
  'Trust recommended'],
 ['Name',
  'Product',
  'Benefit amount',
  'Term',
  'Premium',
  'Payable on',
  'Trust recommended']]

In [1204]:
template_v_df =[]
myset1 = set(way.Field)
#print(myset1)
#pdf_list = list(myset1)
pdf_list = way.Field

for i in range(len(dataframe)):
    tmpl_list=[]
    tmpl_list2 =[]
    out=[]
    tmpl_list = list(dataframe[i].Value)

    #for gg in range(0,len(tmpl_list)):
        #if isinstance(tmpl_list[gg], list) == False:
            #tmpl_list[gg]= list(tmpl_list[gg])


    import itertools
    #lst = [['a'], ['b', 'c'], ['d', ['e', 'f']]]
    outp = list(itertools.product(tmpl_list))
    out = []
    for i in outp:
        temp = []
        for j in i:
            if isinstance(j, list):
                for k in j:
                    temp.append(k)
            else:
                temp.append(j)
        out.append(temp)
    #print(out)

    tmpl_list2  = out

    outp = list(itertools.product(*tmpl_list2))
    out = []
    for i in outp:
        temp = []
        for j in i:
            if isinstance(j, list):
                for k in j:
                    temp.append(k)
            else:
                temp.append(j)
        out.append(temp)
        #print(out)
    template_v_df.append(out) 
    #return template_df
    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [1214]:
template_v_df[0]

[['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  '£',
  'Death',
  'Yes'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  '£',
  'Death',
  'No'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  '£',
  'First death',
  'Yes'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  '£',
  'First death',
  'No'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  'INSERT_4',
  'Death',
  'Yes'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  'INSERT_4',
  'Death',
  'No'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  'INSERT_4',
  'First death',
  'Yes'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
 

In [1184]:
template_v_df[0]

[['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  '£',
  'Death',
  'Yes'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  '£',
  'Death',
  'No'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  '£',
  'First death',
  'Yes'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  '£',
  'First death',
  'No'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  'INSERT_4',
  'Death',
  'Yes'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  'INSERT_4',
  'Death',
  'No'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
  'Zurich',
  '£ii_Field_52_1',
  'ii_Field_62_1 years',
  'INSERT_4',
  'First death',
  'Yes'],
 ['INSERT_3',
  'Aegon',
  'Aviva',
  'HSBC',
 

In [713]:
import pandas as pd 
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process
def checker(wrong_options,correct_options):
    #if len(myset1)== len(dataframe[i].Field):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
           names_array.append(wrong_option)
           ratio_array.append(100)
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_sort_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

In [714]:
df_To_beMatched = (pdf_list)
df_Original_List = (template_df[1][0])

In [715]:
df_To_beMatched

3                       Name
4                    Product
5                Sum assured
6                       Term
7                    Premium
8                   In trust
9    Guaranteed insurability
Name: Field, dtype: object

In [716]:
df_Original_List

['Name',
 'Product',
 'Sum assured',
 'Term',
 'Premium',
 'In trust',
 'Guaranteed insurability option available']

In [717]:
str2Match = (df_To_beMatched)
strOptions =(df_Original_List)

In [718]:
name_match,ratio_match=checker(df_To_beMatched,df_Original_List)


In [719]:
#strOptions

In [720]:
ratio_match

[100, 100, 100, 100, 100, 100, 73]

In [721]:
#template_v_df
names_matching=[]
ratio_matching=[]
template_match = []
#df_To_beMatched=pd.DataFrame()
#df_Original_List = pd.DataFrame()
for i in range(0,len(template_df)):
    for ii in range(0, len(template_df[i])):
        
        #print(a)
        df_To_beMatched = (pdf_list)
        df_Original_List = (template_df[i][ii])
        #print(df_Original_List)
        #print( df_To_beMatched)
        
        #str2Match = df_To_beMatched[0].fillna("######").tolist()
        #strOptions =df_Original_List[0].fillna("######").tolist()
        if len(myset1)== len(template_df[i][ii]):
            name_match,ratio_match=checker(df_To_beMatched,df_Original_List)
            #print("subgroup of ", i)
            #print(name_match)
            #print(ratio_match)
            #print("______________________________________________________")
        template_match.append(i)
        names_matching.append(name_match)
        ratio_matching.append(ratio_match)
    if len(names_matching)==0:
        print("No Tables Matching Template Found")
    #if #template_v_df


In [508]:
ratio_matching , template_match

([[100, 100, 100, 100, 100, 56, 41],
  [100, 100, 44, 100, 100, 56, 38],
  [100, 100, 100, 100, 100, 100, 73],
  [100, 100, 100, 100, 100, 100, 73],
  [100, 100, 100, 100, 100, 100, 73],
  [100, 100, 100, 100, 100, 100, 73],
  [100, 100, 100, 100, 100, 100, 73],
  [100, 100, 100, 100, 100, 56, 41],
  [100, 100, 100, 100, 100, 100, 73]],
 [0, 0, 1, 2, 3, 4, 5, 6, 7])

In [509]:
def greater(sublist1, sublist2):
    ' Checks if sublist greater based upon number of items in one greater than the other '
    cnt = sum(1 for x, y in zip(sorted(sublist1), sorted(sublist2)) if x > y)
    l = len(sublist1)
    
    return cnt > l//2  # True when over half of the items are greater
   
def equal(sublist1, sublist2):
    ' sublist are equal if all terms are equal'
    return sorted(sublist1) == sorted(sublist2)
    
def find_max(lst):
    ' Find all items which are greater than or equal to all others based upon greater or equal functions '
    maxes = [lst[0]]
    
    for i in lst[1:]:
        if greater(maxes[-1], i):
            # Items in max are greater than current
            continue
        elif equal(maxes[-1], i):
            # items in max are equal to current
            maxes.append(i)
        else:
            # current item is greater
            maxes = [i]
   
    return maxes

In [517]:
ratio_matching

[[100, 100, 100, 100, 100, 56, 41],
 [100, 100, 44, 100, 100, 56, 38],
 [100, 100, 100, 100, 100, 100, 73],
 [100, 100, 100, 100, 100, 100, 73],
 [100, 100, 100, 100, 100, 100, 73],
 [100, 100, 100, 100, 100, 100, 73],
 [100, 100, 100, 100, 100, 100, 73],
 [100, 100, 100, 100, 100, 56, 41],
 [100, 100, 100, 100, 100, 100, 73]]

In [518]:
max(ratio_matching)

[100, 100, 100, 100, 100, 100, 73]

In [524]:
template_v_df

[[['INSERT_3',
   'Aegon',
   '£ii_Field_52_1',
   'ii_Field_62_1 years',
   '£',
   'Death',
   'Field_6'],
  ['INSERT_3',
   'Aegon',
   '£ii_Field_52_1',
   'ii_Field_62_1 years',
   '£',
   'First death',
   'Field_6'],
  ['INSERT_3',
   'Aegon',
   '£ii_Field_52_1',
   'to age ii_Field_63_2',
   '£',
   'Death',
   'Field_6'],
  ['INSERT_3',
   'Aegon',
   '£ii_Field_52_1',
   'to age ii_Field_63_2',
   '£',
   'First death',
   'Field_6'],
  ['INSERT_3',
   'Aegon',
   '£ii_Field_52_1',
   'Whole of life',
   '£',
   'Death',
   'Field_6'],
  ['INSERT_3',
   'Aegon',
   '£ii_Field_52_1',
   'Whole of life',
   '£',
   'First death',
   'Field_6'],
  ['INSERT_3',
   'Aegon',
   '£ii_Field_53_2 a month',
   'ii_Field_62_1 years',
   '£',
   'Death',
   'Field_6'],
  ['INSERT_3',
   'Aegon',
   '£ii_Field_53_2 a month',
   'ii_Field_62_1 years',
   '£',
   'First death',
   'Field_6'],
  ['INSERT_3',
   'Aegon',
   '£ii_Field_53_2 a month',
   'to age ii_Field_63_2',
   '£',
   'Dea

In [522]:
its_match = max(ratio_matching)
names_v_matching=[]
ratio_v_matching=[]
pdf_v = way.Value
indices = [i for i, x in enumerate(ratio_matching) if x == its_match]
print(indices)
if len(indices)>1:
    for i in indices:
        j = template_match[i]
        print( "_______________" , j, "________________")
        for ii in range(0, len(template_v_df[j])):
            print (template_v_df[j])
            print("+++++++++++++++++++++++++++++++++++++++++++++++++++")
        
        #print(a)
            df_To_beMatched = (pdf_v)
            df_Original_List = (template_v_df[j][ii])
            print(df_Original_List)
            print( df_To_beMatched)

            #str2Match = df_To_beMatched[0].fillna("######").tolist()
            #strOptions =df_Original_List[0].fillna("######").tolist()
            #if len(myset1)== len(template_df[i][ii]):
            name_match,ratio_match=checker(df_To_beMatched,df_Original_List)
            print(name_match)
            print(ratio_match)
            print("______________________________________________________")

        names_v_matching.append(name_match)
        ratio_v_matching.append(ratio_match)
    if len(names_matching)==0:
        print("No Tables Matching Template Found")

[2, 3, 4, 5, 6, 8]
_______________ 1 ________________
[['INSERT_9', 'INSERT_10', '£', 'INSERT_13', '£', 'Yes', 'Field_18'], ['INSERT_9', 'INSERT_10', '£', 'INSERT_13', '£', 'No', 'Field_18']]
+++++++++++++++++++++++++++++++++++++++++++++++++++
['INSERT_9', 'INSERT_10', '£', 'INSERT_13', '£', 'Yes', 'Field_18']
3                        Elnaz Nobari
4    potatoe Level Life Cover tomatoe
5                                 £13
6              200 years 10 millenium
7                        £400 a month
8                                 Yes
9                                  No
Name: Value, dtype: object
['INSERT_9', 'INSERT_9', 'INSERT_13', 'Yes', 'INSERT_9', 'Yes', 'INSERT_9']
[20, 15, 36, 24, 21, 100, 20]
______________________________________________________
[['INSERT_9', 'INSERT_10', '£', 'INSERT_13', '£', 'Yes', 'Field_18'], ['INSERT_9', 'INSERT_10', '£', 'INSERT_13', '£', 'No', 'Field_18']]
+++++++++++++++++++++++++++++++++++++++++++++++++++
['INSERT_9', 'INSERT_10', '£', 'INSERT_13', 

In [523]:
ratio_v_matching

[[29, 15, 36, 20, 21, 18, 100],
 [20, 15, 20, 8, 20, 29, 18],
 [19, 15, 13, 19, 10, 17, 18],
 [24, 26, 9, 19, 20, 17, 18],
 [29, 26, 18, 19, 20, 17, 100],
 [29, 14, 33, 13, 19, 18, 100]]

In [226]:
sorted_list = sorted(ratio_matching, key=lambda x: max(x[0:]), reverse=True)

# slice first 3 lists (to get the "top" 3 max values)
sliced_list = sorted_list[:3]

print(sliced_list)  

[[100, 100, 44, 100, 100, 56, 38], [100, 100, 100, 100, 100, 100, 73], [100, 100, 100, 100, 100, 100, 73]]


In [960]:
wrd_tbl_0 = pd.DataFrame(k_all_tbl[0])
wrd_tbl_0.columns = ['Field','Value']
#wrd_tbl_0[0] = wrd_tbl_0[0].str.get(0)
#wrd_tbl_0[0] = wrd_tbl_0[0].str.get(0)

In [827]:
#loop to remove all lists from inside dataframe and replace with values 
for cols in wrd_tbl_0:
    for irow in range(0,len(wrd_tbl_0[cols])):
        while isinstance(wrd_tbl_0[cols][irow], list) == True:
            wrd_tbl_0[cols][irow] = wrd_tbl_0[cols][irow][0]
            continue
            if isinstance(wrd_tbl_0[cols][irow], list) == False :
                break
                
        #while type(row) != str:
           # row = row[0]
          #i += 1

In [891]:
len(reviews)

9

In [925]:
way

,Field,Value
3,Name,Elnaz Nobari
4,Product,potatoe Level Life Cover tomatoe
5,Sum assured,£13
6,Term,200 years 10 millenium
7,Premium,£400 a month
8,In trust,Yes
9,Guaranteed insurability,No


In [945]:
way.Field[3]

'Name'

In [946]:
fuzz.ratio(dataframe[0].Field[0], way.Field[3])

100

In [218]:
import xmltodict

with open(soup) as fd:
    doc = xmltodict.parse(fd.read())

TypeError: expected str, bytes or os.PathLike object, not BeautifulSoup

In [357]:
data = []
for label in soup.select('comboBox'):
    data.append({ label.text.strip(): label.find_next_sibling().text.strip() })

AttributeError: 'NoneType' object has no attribute 'text'

In [221]:
for product_div in soup.find_all('w', {'w': 'value'}):
    product_tag = product_div.find('listItem')
    if product_tag:
        print (product_tag.attrs['id'])

In [227]:
all_tags = [tag.name for tag in soup.find_all()]
myset = set(all_tags)
print(myset)

{'tblW', 'color', 'pStyle', 'tblGrid', 'r', 'tc', 'sdtEndPr', 'id', 'listItem', 'b', 'tcPr', 'tblLook', 'vertAlign', 'pPr', 'lastRenderedPageBreak', 'rStyle', 'tblStyle', 'docPart', 'sdt', 'pgSz', 'rPr', 'tcBorders', 'showingPlcHdr', 'tblBorders', 'insideV', 'placeholder', 'tr', 'document', 'rFonts', 'sectPr', 'pgMar', 'tcW', 't', 'p', 'cols', 'tbl', 'body', 'tblPr', 'left', 'gridCol', 'shd', 'trHeight', 'br', 'docGrid', 'sdtPr', 'comboBox', 'trPr', 'right', 'top', 'bottom', 'iCs', 'repeatingSection', 'sdtContent', 'repeatingSectionItem', 'highlight'}


In [407]:
head_tag = soup.sdt
head_tag
for child in head_tag.children:
    k2 = (child, "_____________________________")

In [418]:
#soup.find_all('sdt')[6].contents[2]['w:t']#'t')

dali =[]
mozart = []
beeth = []
for i in range(0,len(soup.findAll('sdt'))):
    #try:
        lolz = soup.findAll('sdt')[i].contents[2]
        dali = lolz("t")
        #beeth = [num["w:t"] for num in lolz]
        mozart.append(dali)

In [420]:
#soup.findAll('sdt')[0].contents[2]
for row in tr:
    print(row.find('td', {'class': 'price'}).text)

<w:sdtContent><w:sdt><w:sdtPr><w:rPr><w:b/></w:rPr><w:id w:val="2052105906"/><w:placeholder><w:docPart w:val="FA5837C7B0AF452AA8A6CBBA1D252A1B"/></w:placeholder><w15:repeatingSectionItem/></w:sdtPr><w:sdtEndPr><w:rPr><w:b w:val="0"/></w:rPr></w:sdtEndPr><w:sdtContent><w:tbl><w:tblPr><w:tblStyle w:val="TableGrid"/><w:tblW w:type="auto" w:w="0"/><w:tblBorders><w:top w:color="auto" w:space="0" w:sz="0" w:val="none"/><w:left w:color="auto" w:space="0" w:sz="0" w:val="none"/><w:bottom w:color="auto" w:space="0" w:sz="0" w:val="none"/><w:right w:color="auto" w:space="0" w:sz="0" w:val="none"/><w:insideV w:color="auto" w:space="0" w:sz="0" w:val="none"/></w:tblBorders><w:tblLook w:firstColumn="1" w:firstRow="1" w:lastColumn="0" w:lastRow="0" w:noHBand="0" w:noVBand="1" w:val="04A0"/></w:tblPr><w:tblGrid><w:gridCol w:w="2589"/><w:gridCol w:w="6437"/></w:tblGrid><w:tr w14:paraId="06FE69F0" w14:textId="77777777" w:rsidR="00F01ED7" w:rsidRPr="002E1A73" w:rsidTr="00DB6E7E"><w:tc><w:tcPr><w:tcW w:t

In [429]:
#mozart

In [716]:
#soup.find_all('sdt')[3].contents

#dali =[]
#mozart = []
#beeth = []
#for i in range(3,len(soup.findAll('sdt'))):
    #try:
        #lolz = soup.findAll('comboBox')[i].contents
        #dali = [num["w:value"] for num in lolz]
        #beeth = [num["w:t"] for num in lolz]
        #mozart.append([beeth, dali])

In [335]:
i_tag = soup.find_all('comboBox')[11]

for i in soup.find_all('comboBox'):
    #print(child)
    my_text = str(i.previousSibling).strip()
    print(my_text, "-------------------------") 

In [356]:
import re
maxdepth = 1
indent_size = 2
x = (soup.prettify())

x2 = xmltodict.parse(x, process_namespaces=True)

for key in x2:
   k =(json.dumps(x2[key], indent=4, default=str))

In [270]:
print(head_tag)

None


In [315]:
t1 = soup.findAll('comboBox')[90].contents#['listItem']
#title_tag = t1[0]
t1

[<w:listItem w:value="Choose an item."/>,
 <w:listItem w:displayText="the premiums are the lowest of all the companies on our list of recommended insurers." w:value="the premiums are the lowest of all the companies on our list of recommended insurers."/>,
 <w:listItem w:displayText="the features and benefits of the policy meet your specific requirements and the premiums are the lowest of all the companies on our list of recommended insurers." w:value="the features and benefits of the policy meet your specific requirements and the premiums are the lowest of all the companies on our list of recommended insurers."/>,
 <w:listItem w:displayText="the features and benefits of the policy meet your specific requirements." w:value="the features and benefits of the policy meet your specific requirements."/>,
 <w:listItem w:displayText="ii_Field_995_1 other reason." w:value="ii_Field_995_1 other reason."/>]

In [324]:
dali =[]
mozart = []
for i in range(0,len(soup.findAll('comboBox'))):
        lolz = soup.findAll('comboBox')[i].contents
        dali = [num["w:value"] for num in lolz]
        mozart.append(dali)
    #t1 = soup.findAll('comboBox')[i].contents
    #for j in range(0, len(soup.findAll('comboBox')[i].contents)):
        #for ii in in range(1, len(soup.findAll('comboBox')[i].contents[j])):
   
    
        
    

In [325]:
mozart

[['Choose an item.', 'Aegon', 'Aviva', 'HSBC', 'Zurich'],
 ['Choose an item.',
  'Level Life Cover',
  'Family Income Benefit',
  'Whole of Life Cover'],
 ['Choose an item.', 'Sum assured', 'Benefit amount'],
 ['Choose an item.', '£ii_Field_52_1', '£ii_Field_53_2 a month'],
 ['Choose an item.',
  'ii_Field_62_1 years',
  'to age ii_Field_63_2',
  'Whole of life'],
 ['Choose an item.', 'Death', 'First death'],
 ['Choose an item.', 'Yes', 'No'],
 ['Choose an item.', 'meet', 'partially meet'],
 ['Choose an item.',
  'replace your earned income',
  'provide a lump sum',
  'provide a monthly income'],
 ['Choose an item.', 'you die', 'either of you dies'],
 ['Choose an item.', 'their', 'his', 'her'],
 ['Choose an item.', 'their', 'his', 'her'],
 ['Choose an item.', 'sums assured and terms', 'sums assured', 'terms'],
 ['Choose an item.',
  'as the total cost is lower',
  'as you feel a single policy is more convenient, despite costing more',
  'because ii_Field_154_1'],
 ['Choose an item.', '

In [289]:
len(soup.findAll('comboBox'))

91

In [287]:
len(t1)

5

In [317]:
t1[1]

<w:listItem w:displayText="the premiums are the lowest of all the companies on our list of recommended insurers." w:value="the premiums are the lowest of all the companies on our list of recommended insurers."/>

In [255]:
t1.attrs

{}

In [251]:
soup.comboBox.lisItem

In [243]:
t = soup.findAll('listItem')[2]
type(t)
t.text
type(t.attrs)
t.attrs #['w:value']

{'w:displayText': 'Aviva', 'w:value': 'Aviva'}

In [ ]:
links = soup.find_all('w')

In [ ]:
for link in links:
    print(link.attrs['listItem'])

In [ ]:
soup.listItem.value

In [ ]:
import argparse
from xml.etree import ElementTree


# ------------------------------------------------------------------------------
def reformat(
        input_xml: str,
        output_xml: str,
):
    tree = ElementTree.parse(input_xml)

    # remove extraneous newlines and whitespace from text elements
    for element in tree.getiterator():
        if element.text:
            element.text = element.text.strip()

    # write the updated XML into the annotations output directory
    tree.write(output_xml)


# ------------------------------------------------------------------------------
if __name__ == "__main__":

    # parse the command line arguments
    args_parser = argparse.ArgumentParser()
    args_parser.add_argument(
        "--in",
        required=True,
        type=str,
        help="file path of original XML",
    )
    args_parser.add_argument(
        "--out",
        required=True,
        type=str,
        help="file path of reformatted XML",
    )
    args = vars(args_parser.parse_args())

    reformat(
        args["in"],
        args["out"],
    )

In [ ]:
 #with open(r'Protection recommendations_1.docx', encoding="utf8", errors='ignore') as f:
    #soup5 = BeautifulSoup(f, 'lxml')

In [ ]:
all_tags = [tag.name for tag in soup.find_all()]

In [ ]:
myset = set(all_tags)
print(myset)

In [ ]:
for row in soup.find_all('column'):
    print(row.text)

In [ ]:
import lxml.etree
document = zipfile.ZipFile('Protection recommendations_1.docx')
xml_content = document.read('word/document.xml')
#tree = lxml.etree.parse(xml_content)

In [ ]:
from lxml import etree
from io import BytesIO

myString = xml_content

tree = etree.parse(BytesIO(myString))

In [ ]:
pi = tree.xpath("//processing-instruction()")[0] 

In [ ]:
gegevans

In [428]:
#soup.findAll('tbl')
#print (soup.prettify())

In [ ]:
soup.findAll('listItem') 

In [ ]:
#convert to string for if statements
number = str(dropdownlist[0])
job = str(dropdownlist[1])
vehicle = str(dropdownlist[7])

if number == '<w:result w:val="1"/>' :
    dropdownlistdata.append('0,3')
elif number == '<w:result w:val="2"/>' :
    dropdownlistdata.append('1,2')
elif number == '<w:result w:val="3"/>' :
    dropdownlistdata.append('onbekend')
else:
    dropdownlistdata.append('geen')

if job  == '<w:result w:val="1"/>' :
    dropdownlistdata.append('nee')
else:
    dropdownlistdata.append('ja')

if vehicle == '<w:result w:val="1"/>' :
    dropdownlistdata.append('nee')
else:
    dropdownlistdata.append('ja')

#show data
print (alletabellen)
print (dropdownlistdata)

In [ ]:
Sec_title = list(df[2])
Sec_title  = [x for x in Sec_title if str(x) != 'nan']

In [ ]:
hl_title = list (df[1])
hl_title  = [x for x in hl_title if str(x) != 'nan']

In [ ]:
Sec_title

In [ ]:
hl_title

In [ ]:
import os
import sys
scriptpath = "/home/jupyter/elmodules"
sys.path.append(os.path.abspath(scriptpath))

In [ ]:
import data_func

In [ ]:
import PyPDF2
#import data_func
import csv

reader = PyPDF2.PdfFileReader('/home/jupyter/FPR_281020_3.pdf')

print(reader.documentInfo)

num_of_pages = reader.numPages
print('Number of pages: ' + str(num_of_pages))

In [ ]:
title_list = []
pagenum_list = []
title_formatted_list = []
for item in table_of_contents_raw:
        title, pagenum = \
            data_func.split_to_title_and_pagenum(item)
        if title != None:
            title_list.append(title)
            pagenum_list.append(pagenum)
            title_formatted_list.append(
                data_func.convert_title_to_filename(title))
            
# for page_list, we need to add the last page as well
pagenum_list.append(num_of_pages + 1)

In [ ]:

for i in range(1, len(Sec_title)):
    title_formatted = Sec_title[i]
    #page_start = pagenum_list[i] - 1
    #page_end = pagenum_list[i+1] - 2
    
    writer = PyPDF2.PdfFileWriter()

    for page in range(0,num_of_pages + 1):
        writer.addPage(reader.getPage(page))
    
    output_filename = './data/original/pdfs/' + title_formatted + '.pdf'

    with open(output_filename, 'wb') as output:
        writer.write(output)